In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
#from PIL import Image
import os
from keras import layers


#Data is downloaded into the container via shellscript. We believe this is more efficient
traindirectory="./archive/FaceMaskDataset/Train"
testdirectory="./archive/FaceMaskDataset/Test"
validationdirectory="./archive/FaceMaskDataset/Validation"
image_size=224
TrainData=keras.utils.image_dataset_from_directory(traindirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))
TestData=keras.utils.image_dataset_from_directory(testdirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))
ValidationData=keras.utils.image_dataset_from_directory(validationdirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))

from datetime import datetime
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

early_stop= tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2
)

def build_model(num_classes, IMG_SIZE):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    #x=inputs
    model = keras.applications.EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", dtype='float32', name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

model=build_model(2,224)
model.fit(TrainData, epochs=1, validation_data=TestData)

Found 10000 files belonging to 2 classes.
Found 992 files belonging to 2 classes.
Found 800 files belonging to 2 classes.
313/313 [==============================] - 62s 158ms/step - loss: 0.0790 - accuracy: 0.9906 - val_loss: 0.0356 - val_accuracy: 0.9970


In [2]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

q_aware_model.fit(TrainData, epochs=2, validation_data=TestData)

C:\Users\seang\anaconda3\envs\tf_gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


RuntimeError: Layer rescaling:<class 'keras.layers.preprocessing.image_preprocessing.Rescaling'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.